In [ ]:
#                                              ::  Angelic  ::

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras.optimizers import Adam



#-------------------------------------------------------start layer RUDRA--------------------------------------------------
# Generator model
def build_generator(latent_dim, output_shape):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.5))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.5))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(output_shape), activation='linear'))
    model.add(Reshape(output_shape))
    return model

# Discriminator model
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.8))
    model.add(Dense(1, activation='linear'))
    return model

# CNN-LSTM hybrid model
def build_cnn_lstm(latent_dim, input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(200))
    model.add(Dense(1, activation='linear'))
    return model

# GAN model
def build_gan(generator, discriminator, input_shape):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(Reshape((input_shape[0], input_shape[1])))
    model.add(discriminator)
    return model

#-------------------------------------------------------end layer RUDRA--------------------------------------------------
#----------------------------------------------------------Cheers!-------------------------------------------------------
#---------------------------------------------------------Subhadeep------------------------------------------------------

# Potential and example data loading (replace with your own data loading logic)
file_path = r'C:\Users\Subhadeep Maishal\Music\JUP\test_data\CBN_INPUT.csv'
df = pd.read_csv(file_path, parse_dates=['DATE'])
features = df[['AAO', 'AO', 'DMI', 'NAO', 'ONI', 'PDO', 'SOI', 'TNA', 'TSA', 'WHWP', 'WP']]
target = df['MEIv2']
features = (features - features.min()) / (features.max() - features.min())
np.random.seed(1000)

# Build and compile the discriminator
discriminator = build_discriminator((features.shape[1], 1))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5), metrics=['accuracy'])

# Build and compile the generator
latent_dim = 100
generator = build_generator(latent_dim, (features.shape[1], 1))
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5))

# Build and compile the CNN-LSTM hybrid model
cnn_lstm = build_cnn_lstm(latent_dim, (features.shape[1], 1))
cnn_lstm.compile(loss='mean_squared_error', optimizer=Adam(0.2, 0.5))

# Build and compile the GAN
discriminator.trainable = False
gan = build_gan(generator, discriminator, (features.shape[1], 1))
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5))

# Training GAN
epochs = 10000
batch_size = 32

half_batch = int(batch_size / 2)

for epoch in range(epochs):
    idx = np.random.randint(0, features.shape[0], half_batch)
    real_data = features.iloc[idx].values  # Convert to numpy array

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    generated_data = generator.predict(noise)

    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, valid_labels)

    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

# Generate data useing CNN-LSTM hybrid model >>
num_samples = 420
noise = np.random.normal(0, 1, (num_samples, latent_dim))
generated_data = generator.predict(noise)
synthetic_data = cnn_lstm.predict(generated_data.reshape((num_samples, features.shape[1], 1)))

# Plot actual and generated data
plt.figure(figsize=(12, 8))
plt.plot(df['DATE'], target, label='Actual', marker='o', color='black')
plt.plot(df['DATE'].iloc[-num_samples:], synthetic_data[:, 0], label='Generated', marker='o', color='blue')
plt.xlabel('Date', fontsize=14)
plt.ylabel('MEIv2 Values', fontsize=14)
plt.title('Actual vs Generated MEIv2 Values Over Time', fontsize=16)
plt.legend(fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


1/1 [==============================] - 0s 100ms/step
0 [D loss: 6.5556745529174805 | D accuracy: 12.5] [G loss: 8.194503784179688]
1/1 [==============================] - 0s 31ms/step
100 [D loss: 3.812309741973877 | D accuracy: 75.0] [G loss: 8.194503784179688]
1/1 [==============================] - 0s 38ms/step
200 [D loss: 3.812309741973877 | D accuracy: 75.0] [G loss: 7.7124738693237305]
1/1 [==============================] - 0s 16ms/step
300 [D loss: 4.765387058258057 | D accuracy: 68.75] [G loss: 7.23044490814209]
1/1 [==============================] - 0s 16ms/step
400 [D loss: 3.335770845413208 | D accuracy: 78.125] [G loss: 8.194503784179688]
1/1 [==============================] - 0s 16ms/step
500 [D loss: 1.906154751777649 | D accuracy: 87.5] [G loss: 7.712474346160889]
1/1 [==============================] - 0s 16ms/step
600 [D loss: 4.288848400115967 | D accuracy: 71.875] [G loss: 7.712474346160889]
1/1 [==============================] - 0s 16ms/step
700 [D loss: 2.85923218727

In [ ]:
#                                             :) :) :)